---
title: Python property 原理解析与复现
tags: 小书匠,python,property,descriptor,__set__,__get__,setter
grammar_cjkRuby: true
renderNumberedHeading: true,
---

[toc]

# Python property 原理解析与复现

In [75]:
class People(object):
    def __init__(self, name):
        self._name = name
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

people = People('Edward')
print(people.name)  # Tom
people.name = 'Tony'
print(people.name)  # Tony

Edward
Tony


## Descriptor

descriptor 的作用是让一个类被作为属性调用时返回某个函数

In [20]:
class Ten:
    def __get__(self, obj, objtype=None):
        print(obj)
        return 10
    
class A:
    x = 5                       # Regular class attribute
    y = Ten()  

a = A()
print(a.x)
print(a.y)

5
10


上面这个示例很好的说明了 Descriptor 的作用。它用起来和普通的值类似，但是可以运行一个函数。

## Property

1. 我们用 property 修饰 name 之后，发现调用 name 时会运行一个函数。这和 descriptor 的功能类似。因此我们可以认为，property 修饰过的 name 应该是一个 descriptor。
2. descriptor 是一个类的实例，而非函数，因此我们的 property 也应该使用类作为装饰器。关于类装饰器，可以参考 [Python Decorator 装饰器 wrapper](https://app.yinxiang.com/shard/s54/nl/22483756/fcc4cfd5-180f-43ad-bf0c-5c8131a28773/)，只不过，这里的装饰器不实现 `__call__` 方法，而是要实现 `__get__` 方法。

In [66]:
class Property:
    
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner=None):
        return self.func(instance)
    
class People(object):
    def __init__(self, name):
        self._name = name
    
    @Property
    def name(self):
        return self._name

people = People('Edward')
print("name: {} _name: {}".format(people.name, people._name))  # Edward Edward


name: Edward _name: Edward


## 实现 setter

In [45]:
class People(object):
    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

people = People('Edward')
print(people.name)  # Tom
people.name = 'Tony'
print(people.name)  # Tony

Edward
Tony


在 property 中，我们发现 `name.setter` 也是一个装饰器，用来接受一个另一个函数。而且也是通过属性调用的，因此也是一个 descriptor。
在这个过程中，我们发现又两个函数：一个是 `__get__` 中运行的，一个是 `__set__` 中运行的，因此我们需要用两个函数来区分，我们用 `fget` 和 `fset` 来区分。

同时，我们希望 name.setter 是一个接受 fset 参数的函数，并且也是一个 descriptor，我们可以让 name.setter 返回一个 Property 对象。

我们的过程是这样的：

1. 首先调用 `@Property`，创建一个 descriptor，这个 descriptor 的 `fget` 属性被赋值了。
2. 然后调用 `@name.setter` 时返回一个新的 descriptor，这个 descriptor 一方面要有第一个 descriptor 的功能，即新的 descriptor 的 fget 方法要和第一个 descriptor 的 fget 方法相同，另一方面新的 descriptor 还接受一个 fset 参数。

In [69]:
class Property:
    
    def __init__(self, fget=None, fset=None):
        self.fget = fget
        self.fset = fset
        self.setter = lambda fset: Property(fget=fget, fset=fset)
        
    def __get__(self, instance, owner=None):
        return self.fget(instance)
    
    def __set__(self, instance, val):
        self.fset(instance, val)
        
    def setter(self, fset):
        return Property(fget=self.fget, fset=fset)
    
class People(object):
    def __init__(self, name):
        self._name = name
    
    @Property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, val):
        self._name = val + "_hah"

people = People('Edward')
print("name: {} _name: {}".format(people.name, people._name))  # Edward Edward
people.name = 'Tom' 
print("name: {} _name: {}".format(people.name, people._name))  # Tom Tom
people.name = 'Yuki' 
print("name: {} _name: {}".format(people.name, people._name))  # Yuki

name: Edward _name: Edward
name: Tom_hah _name: Tom_hah
name: Yuki_hah _name: Yuki_hah


上面的代码中在 setter 中创建了一个新的 Property 对象，我们也可以在 setter 中给原来的 descriptor 的 fset 方法赋值后返回原来的 Property 对象而不是创建一个新的 Property 对象。

In [76]:
class Property:
    
    def __init__(self, fget=None, fset=None):
        self.fget = fget
        self.fset = fset
        
    def __get__(self, instance, owner=None):
        return self.fget(instance)
    
    def __set__(self, instance, val):
        self.fset(instance, val)
        
    def setter(self, fset):
        self.fset = fset
        return self
    
class People(object):
    def __init__(self, name):
        self._name = name
    
    @Property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, val):
        self._name = val + "_hah"

people = People('Edward')
print("name: {} _name: {}".format(people.name, people._name))  # Edward Edward
people.name = 'Tom' 
print("name: {} _name: {}".format(people.name, people._name))  # Tom Tom
people.name = 'Yuki' 
print("name: {} _name: {}".format(people.name, people._name))  # Yuki

name: Edward _name: Edward
name: Tom_hah _name: Tom_hah
name: Yuki_hah _name: Yuki_hah


# References
- http://localhost:8888/lab/tree/learnPython/Python%20Property.ipynb
- [Python中 property 的实现原理及实现纯 Python 版 - 知乎](https://zhuanlan.zhihu.com/p/142029367#:~:text=%E5%B0%BD%E7%AE%A1%20property%20%E7%9A%84%E5%AE%9E%E7%8E%B0%E6%98%AF%20C%20%E5%AE%9E%E7%8E%B0%EF%BC%8C%E4%BD%86%E4%BB%8D%E4%B8%8D%E5%A6%A8%E7%A2%8D%E6%8E%A2%E7%A9%B6%E5%AE%83%E7%9A%84%E5%AE%9E%E7%8E%B0%E5%8E%9F%E7%90%86%EF%BC%8C%E6%9C%AC%E6%96%87%E6%9C%80%E5%90%8E%E4%B9%9F%E4%BC%9A%E7%BB%99%E5%87%BA%E5%AE%83%E7%9A%84%E7%BA%AF%20Python%20%E7%89%88%E6%9C%AC%E7%9A%84%E5%AE%9E%E7%8E%B0%E3%80%82%E6%8F%8F%E8%BF%B0%E7%AC%A6%E5%AF%B9%E8%B1%A1,%E4%B8%BA%E4%BA%86%E8%83%BD%E5%A4%9F%E5%AE%9E%E7%8E%B0%E8%AE%BF%E9%97%AE%E5%B1%9E%E6%80%A7%E5%B0%B1%E8%B0%83%E7%94%A8%E6%9F%90%E4%B8%AA%E5%87%BD%E6%95%B0%EF%BC%8C%E8%BF%99%E9%87%8C%E5%B0%86%E5%88%A9%E7%94%A8%20%E6%8F%8F%E8%BF%B0%E7%AC%A6%E5%AF%B9%E8%B1%A1%20%E4%BD%9C%E4%B8%BA%E6%9C%AC%E6%96%87%E7%9A%84%E5%AE%9E%E7%8E%B0%E8%B5%B7%E7%82%B9%EF%BC%8C%E5%BD%93%E6%9F%90%E4%B8%AA%E7%B1%BB%E5%AE%9A%E4%B9%89%E4%BA%86%20__get__%20%E6%96%B9%E6%B3%95%E5%90%8E%EF%BC%8C%E9%80%9A%E8%BF%87%E5%85%B6%E6%96%B9%E6%B3%95%E5%90%8D%E7%A7%B0%E5%8F%AF%E4%BB%A5%E7%9B%B4%E6%8E%A5%E8%B0%83%E7%94%A8%20__get__%20%EF%BC%8C%E4%BE%8B%E5%A6%82%EF%BC%9A)
- [Descriptor HowTo Guide — Python 3.9.4 documentation](https://docs.python.org/3/howto/descriptor.html#closing-thoughts)